<a href="https://colab.research.google.com/github/Thealansan/Evidencia2/blob/main/LastMile_Evidencia2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importamos las librerias pandas, numpy y matplotlib respectivamente
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
#Cargar archivo excel desde equipo
from google.colab import files
files.upload()

In [ ]:
#Carga desde un archivo excel sin indice 
data= pd.read_excel('lastmile_delivery_operations_mit_lift_lab.xlsx')

In [ ]:
#Verificamos información del Dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 30 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   _record_id                                     390 non-null    object 
 1   _title                                         380 non-null    object 
 2   _server_updated_at                             390 non-null    object 
 3   _created_by                                    390 non-null    object 
 4   _updated_by                                    390 non-null    object 
 5   _geometry                                      390 non-null    object 
 6   _latitude                                      390 non-null    float64
 7   _longitude                                     390 non-null    float64
 8   arrival_of_the_freight_vehicle                 390 non-null    object 
 9   plates                                         389 non

In [ ]:
#Corroboramos valores nulos 
valores_nulos=data.isnull().sum()
valores_nulos

_record_id                                         0
_title                                            10
_server_updated_at                                 0
_created_by                                        0
_updated_by                                        0
_geometry                                          0
_latitude                                          0
_longitude                                         0
arrival_of_the_freight_vehicle                     0
plates                                             1
company_if_visible                               155
visit_purpose                                      0
type_of_vehicle                                    0
number_of_operators                                0
refrigerated_truck                                 0
type_of_cargo                                      1
picture_of_the_parked_freight_vehicle              0
departure_of_the_freight_vehicle                   0
where_was_the_vehicle_parked                  

In [ ]:
data.head(3)

,_record_id,_title,_server_updated_at,_created_by,_updated_by,_geometry,_latitude,_longitude,arrival_of_the_freight_vehicle,plates,...,used_traffic_cone,vehicles_unloading_door,number_of_available_trolleys,serving_customer,garage_blocking,accident,describe_the_accident,noise,traffic_congestion,maximum_number_of_vehicles_in_the_traffic_jam
0,1e369271-598a-464c-aec7-1913e82dc2b7,78 Avenida Orión Sur\nTlaxcalancingo Puebla 72...,2022-11-27 22:14:32 UTC,Adriel Antonio Huerta Vidal,Adriel Antonio Huerta Vidal,SRID=4326;POINT(-98.2603919 19.0213764),19.021376,-98.260392,15:35,21TFP4,...,no,Rear,0,NaN,no,no,NaN,no,no,NaN
1,4a749c00-7c5f-45cf-8950-aa988a0d41b8,581 Avenida Orión Sur\nTlaxcalancingo Puebla 7...,2022-11-27 22:14:31 UTC,Adriel Antonio Huerta Vidal,Adriel Antonio Huerta Vidal,SRID=4326;POINT(-98.2605677 19.0211113),19.021111,-98.260568,15:12,19TES7,...,no,Rear,0,NaN,no,no,NaN,no,no,NaN
2,c58a450d-fe1f-4a16-a8d8-764ebd8c49a4,4483 Calle Camino Real a Cholula\nSan Bernardi...,2022-11-27 20:12:39 UTC,María Pedraza,María Pedraza,SRID=4326;POINT(-98.2670348883 19.0352214575),19.035221,-98.267035,13:50,Sm49187,...,no,Rear,0,NaN,no,no,NaN,no,yes,NaN


In [ ]:
#Eliminar columnas innecesarias
data2=data.drop(["_record_id", "picture_of_the_parked_freight_vehicle","serving_customer","describe_the_accident","maximum_number_of_vehicles_in_the_traffic_jam"], axis=1) #axis1=columnas y axis0=filas
data2.head()

In [ ]:
data2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   _title                               380 non-null    object 
 1   _server_updated_at                   390 non-null    object 
 2   _created_by                          390 non-null    object 
 3   _updated_by                          390 non-null    object 
 4   _geometry                            390 non-null    object 
 5   _latitude                            390 non-null    float64
 6   _longitude                           390 non-null    float64
 7   arrival_of_the_freight_vehicle       390 non-null    object 
 8   plates                               389 non-null    object 
 9   company_if_visible                   235 non-null    object 
 10  visit_purpose                        390 non-null    object 
 11  type_of_vehicle                 

In [ ]:
#Filtro por columnas, de acuerdo a la información obtenida arriba obtenemos las cuantitativas
cuant=data2.iloc[ : , [12,20]]  #columnas no consecutivas 
cuant

,number_of_operators,number_of_available_trolleys
0,1,0
1,1,0
2,2,0
3,2,0
4,1,0
...,...,...
385,1,0
386,1,0
387,1,1
388,2,0


In [ ]:
#Eliminar columnas innecesarias para crear el subdf de las cualitativas
cualit=data2.drop(["_latitude", "_latitude","number_of_operators","number_of_available_trolleys"], axis=1) #axis1=columnas y axis0=filas
cualit

In [ ]:
#Corroboramos valores nulos 
valores_nulos=cualit.isnull().sum()
valores_nulos
#cualit.info()

_title                                  10
_server_updated_at                       0
_created_by                              0
_updated_by                              0
_geometry                                0
_longitude                               0
arrival_of_the_freight_vehicle           0
plates                                   1
company_if_visible                     155
visit_purpose                            0
type_of_vehicle                          0
refrigerated_truck                       0
type_of_cargo                            1
departure_of_the_freight_vehicle         0
where_was_the_vehicle_parked             0
while_parked_was_the_engine_running      0
used_traffic_cone                        0
vehicles_unloading_door                  0
garage_blocking                          0
accident                                 0
noise                                    0
traffic_congestion                       0
dtype: int64

In [ ]:
#Sustituir valores nulos por valores no nulos hacia adelante "forward fill"("ffill")
cualit["_title"]=cualit["_title"].fillna(method="ffill")
cualit["plates"]=cualit["plates"].fillna(method="bfill")
cualit["type_of_cargo"]=cualit["type_of_cargo"].fillna(method="bfill")
#Sustituir valores nulos por un string en concreto
cualit["company_if_visible"]=cualit["company_if_visible"].fillna("Sin registro")
cualit

In [ ]:
valores_nulos=cualit.isnull().sum()
valores_nulos

_title                                 0
_server_updated_at                     0
_created_by                            0
_updated_by                            0
_geometry                              0
_longitude                             0
arrival_of_the_freight_vehicle         0
plates                                 0
company_if_visible                     0
visit_purpose                          0
type_of_vehicle                        0
refrigerated_truck                     0
type_of_cargo                          0
departure_of_the_freight_vehicle       0
where_was_the_vehicle_parked           0
while_parked_was_the_engine_running    0
used_traffic_cone                      0
vehicles_unloading_door                0
garage_blocking                        0
accident                               0
noise                                  0
traffic_congestion                     0
dtype: int64

In [ ]:
#Método aplicando desviación estandar. Encuentro los valores extremos
y=cuant
Limite_Superior= y.mean() + 3*y.std()
Limite_Inferior= y.mean() - 3*y.std()
print("Limite superior permitido", Limite_Superior)
print("Limite inferior permitido", Limite_Inferior)

Limite superior permitido number_of_operators             4.009477
number_of_available_trolleys    2.962660
dtype: float64
Limite inferior permitido number_of_operators            -0.753067
number_of_available_trolleys   -1.767788
dtype: float64


In [ ]:
#Encontramos Ouliers del Dataframe
outliers= cuant[(y>Limite_Superior)|(y<Limite_Inferior)]
outliers

,number_of_operators,number_of_available_trolleys
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
385,NaN,NaN
386,NaN,NaN
387,NaN,NaN
388,NaN,NaN


In [ ]:
valores_nulos=outliers.isnull().sum()
valores_nulos

number_of_operators             387
number_of_available_trolleys    386
dtype: int64

Ya que al hacer la operación nos da un numéro demasiado elevado, casi el total de entradas, no se hará el procedimiento de outliers

In [ ]:
# Unimos la columna 'indice_tiempo' con el dataframe
lonnylat=data2.iloc[ : , [5,6]]
cuant2 = pd.concat([lonnylat, cuant], axis=1)
cuant2

,_latitude,_longitude,number_of_operators,number_of_available_trolleys
0,19.021376,-98.260392,1,0
1,19.021111,-98.260568,1,0
2,19.035221,-98.267035,2,0
3,19.081790,-98.298561,2,0
4,19.081771,-98.298589,1,0
...,...,...,...,...
385,19.043612,-98.194933,1,0
386,19.043607,-98.194856,1,0
387,19.043336,-98.194503,1,1
388,19.043576,-98.194873,2,0


In [ ]:
LMLEvidencia = pd.concat([cuant2, cualit], axis=1)
LMLEvidencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   _latitude                            390 non-null    float64
 1   _longitude                           390 non-null    float64
 2   number_of_operators                  390 non-null    int64  
 3   number_of_available_trolleys         390 non-null    int64  
 4   _title                               390 non-null    object 
 5   _server_updated_at                   390 non-null    object 
 6   _created_by                          390 non-null    object 
 7   _updated_by                          390 non-null    object 
 8   _geometry                            390 non-null    object 
 9   _longitude                           390 non-null    float64
 10  arrival_of_the_freight_vehicle       390 non-null    object 
 11  plates                          

In [ ]:
#Corroboramos valores nulos del dataframe LIMPIO
valores_nulos=LMLEvidencia.isnull().sum().sum()
valores_nulos

0

In [ ]:
#Convertir DataFrame a CSV
LMLEvidencia.to_csv("LMLEvidencia.csv")

In [ ]:
#descargar archivo filtrado en csv
from google.colab import files 
files.download("LMLEvidencia.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Se finaliza la descarga con un dataframe más limpio, con menos variables desgraciadamente y dificil de tratar sin embargo listo en cuanto al filtrado